# Modul sqlite3

In [1]:
import sqlite3.dbapi2 as sqlite

## 1. Vytvorenie spokenia a kurzora 

In [2]:
conn = sqlite.connect('vzdelavanie.sqlite')
curs = conn.cursor()

## 2. Navrh struktury tabulky

In [3]:
curs.execute("CREATE TABLE programy(id_programu INTEGER PRIMARY KEY, nazov TEXT, forma TEXT, stupen INTEGER);")

## 3. Naplnenie tabulky

In [4]:
curs.execute("INSERT INTO programy VALUES (120, 'informatika', 'denna', 1)")
conn.commit()

In [6]:
pr = [(121, 'pocitacove inzinierstvo', 'denna', 1),
      (122, 'manazment', 'denna', 1),
      (123, 'manazment', 'externa', 1),
      (220, 'informacne systemy', 'denna', 2),
      (221, 'pocitacove inzinierstvo', 'denna', 2),
      (232, 'informacny manazment', 'denna', 2),
      (233, 'aplikovane sietove inzinierstvo', 'denna', 2),
      (234, 'inteligentne informacne systemy', 'denna', 2),
      (235, 'informacny manazment', 'externa', 2)]

In [7]:
curs.executemany("INSERT INTO programy(id_programu, nazov, forma, stupen) VALUES (?, ?, ?, ?)", pr)
conn.commit()

## 4. Vyber udajov z tabulky

In [9]:
# Vsetko
curs.execute("SELECT * FROM programy")
curs.fetchall()

[(120, 'informatika', 'denna', 1),
 (121, 'pocitacove inzinierstvo', 'denna', 1),
 (122, 'manazment', 'denna', 1),
 (123, 'manazment', 'externa', 1),
 (220, 'informacne systemy', 'denna', 2),
 (221, 'pocitacove inzinierstvo', 'denna', 2),
 (232, 'informacny manazment', 'denna', 2),
 (233, 'aplikovane sietove inzinierstvo', 'denna', 2),
 (234, 'inteligentne informacne systemy', 'denna', 2),
 (235, 'informacny manazment', 'externa', 2)]

In [12]:
# Nazvy programov dennej formy bakalarskeho studia
curs.execute("SELECT nazov FROM programy WHERE stupen = 1 AND forma = 'denna'")
curs.fetchall()

[('informatika',), ('pocitacove inzinierstvo',), ('manazment',)]

In [14]:
# Pocet studijmych programov v jednotlivych stupnoch
curs.execute("SELECT stupen, COUNT(stupen) FROM programy GROUP BY stupen")
curs.fetchall()

[(1, 4), (2, 6)]

## 5. Prepojenie tabuliek
Vytvorime dalsiu tabulku studentov s nasledujucov strukturov: id studenta, meno, priezvisko, rocnik, program

In [23]:
curs.execute("CREATE TABLE studenti(id_studenta integer PRIMARY KEY, meno_studenta text, priezvisko text NOT NULL, rocnik integer, program integer, foreign key(program) references programy(id_programu))")

In [17]:
st = [(3791, 'Peter', 'Pan', 1, 120),
      (6134, 'Jan', 'Hrasko', 2, 120),
      (7543, 'Lena', 'Pysna', 1, 122),
      (5791, 'Maria', 'Kratka', 1, 123),
      (7410, 'Jozef', 'Mrkvicka', 1, 221),
      (9632, 'Eva', 'Benova', 2, 235)]

In [24]:
curs.executemany("INSERT INTO studenti(id_studenta, meno_studenta, priezvisko, rocnik, program) VALUES(?, ?, ?, ?, ?)", st)
conn.commit()

In [25]:
curs.execute("SELECT * FROM studenti")
curs.fetchall()

[(3791, 'Peter', 'Pan', 1, 120),
 (5791, 'Maria', 'Kratka', 1, 123),
 (6134, 'Jan', 'Hrasko', 2, 120),
 (7410, 'Jozef', 'Mrkvicka', 1, 221),
 (7543, 'Lena', 'Pysna', 1, 122),
 (9632, 'Eva', 'Benova', 2, 235)]

In [4]:
# Vybrat vsetkych studentov a zora podla abecedy
curs.execute("SELECT priezvisko, meno_studenta FROM studenti ORDER BY priezvisko")
curs.fetchall()

[('Benova', 'Eva'),
 ('Hrasko', 'Jan'),
 ('Kratka', 'Maria'),
 ('Mrkvicka', 'Jozef'),
 ('Pan', 'Peter'),
 ('Pysna', 'Lena')]

In [6]:
# Vybrat studentov informatiky
curs.execute("SELECT s.priezvisko, s.meno_studenta FROM studenti s, programy p WHERE p.nazov='informatika' AND s.program=p.id_programu")
curs.fetchall()

[('Pan', 'Peter'), ('Hrasko', 'Jan')]

In [9]:
# Vybrat studentov externeho studia
curs.execute("SELECT s.priezvisko, s.meno_studenta FROM studenti s, programy p WHERE p.forma='externa' AND s.program=p.id_programu")
curs.fetchall()

[('Kratka', 'Maria'), ('Benova', 'Eva')]

In [20]:
# Pocet studentov denneho bakalarskeho studia
curs.execute("SELECT COUNT(s.id_studenta) FROM studenti s, programy p WHERE p.forma='denna' AND p.stupen=1 AND s.program=p.id_programu")
curs.fetchall()

[(3,)]

In [22]:
# pocty studentov v jednotlivych programoch externeho studia
curs.execute("SELECT p.nazov, COUNT(s.id_studenta) FROM studenti s, programy p WHERE p.forma='externa' AND s.program=p.id_programu GROUP BY p.nazov")
curs.fetchall()

[('informacny manazment', 1), ('manazment', 1)]

## 5. Uprava existujucich udajov

In [27]:
# zmenit studentovi Panovi rocnik na 3
curs.execute("UPDATE studenti SET rocnik=3 WHERE priezvisko='Pan'")
conn.commit()

In [28]:
curs.execute("SELECT * FROM studenti")
curs.fetchall()

[(3791, 'Peter', 'Pan', 3, 120),
 (5791, 'Maria', 'Kratka', 1, 123),
 (6134, 'Jan', 'Hrasko', 2, 120),
 (7410, 'Jozef', 'Mrkvicka', 1, 221),
 (7543, 'Lena', 'Pysna', 1, 122),
 (9632, 'Eva', 'Benova', 2, 235)]

In [ ]:
# Vymazat studentov bakalarskehjo studia
curs.execute("DELETE FROM studenti WHERE program LIKE '1%'")
conn.commit()

## 6. Ukoncenie

In [26]:
curs.close()
conn.close()